In [1]:
import requests
from bs4 import BeautifulSoup
import json

# News API example
API_KEY = 'da2dc6ff6d1544609b318ac2482525fd'
query = 'compliance AND regulatory AND finance'
url = f'https://newsapi.org/v2/everything?q={query}&apiKey={API_KEY}'

response = requests.get(url)
data = response.json()


def get_full_article_text(url):
    """
    Attempts to retrieve the full text of an article by scraping its URL.
    Returns None if scraping is unsuccessful.
    """
    try:
        article_response = requests.get(url, timeout=5)
        soup = BeautifulSoup(article_response.text, 'html.parser')

        # This is a very naive approach. The actual selector might vary by site.
        paragraphs = soup.find_all('p')
        full_text = ' '.join(p.get_text() for p in paragraphs)
        return full_text.strip()
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return None
    

articles = []
for article in data.get("articles", []):
    title = article.get("title")
    description = article.get("description")
    article_url = article.get("url")
    
    # Attempt to scrape the full article
    full_text = get_full_article_text(article_url)
    
    articles.append({
        "title": title,
        "description": description,
        "url": article_url,
        "content": article.get("content"),  # snippet from NewsAPI
        "full_text": full_text              # best-effort full text from scraping
    })

with open('compliance_articles_full.json', 'w') as f:
    json.dump(articles, f, indent=2)

print("Data saved to compliance_articles.json")

Data saved to compliance_articles.json
